In [1]:
# !pip install netCDF4 # if not installed
#!pip install meshio # if meshio is not installed
#!pip install math # if math is not installed
#!pip install numpy # if numpy is not installed

In [2]:
import sys
sys.path.insert(1,'/Applications/pygplates_rev28_python38_MacOS64/')
# please download the suitable pygplates from the link below:
# https://sourceforge.net/projects/gplates/files/pygplates/beta-revision-28/
# valid as of 27 May 2022

import pygplates
import os
import numpy as np    
# import csv
import meshio
import math
# from pylab import *
import netCDF4 as nc
from netCDF4 import Dataset 
# import matplotlib.pyplot as plt

# Reconstruct Kimberlites, Export them as Points (PolyVertex) on the Surface (*vtu files)

In [10]:
def find(lst, a, b):
    return [i for i, x in enumerate(lst) if x<a and x>b]


root='.'
NNRFiles=root+'/NNR_InFiles/'

rotation_file1 = NNRFiles + '1000-410_rotations-NNR.rot'
rotation_file2 = NNRFiles + 'Global_EB_410-250Ma_GK07_2017-NNR.rot'
rotation_file3 = NNRFiles + 'Global_EB_250-0Ma_GK07_2017-NNR.rot'
rotation_file4 = NNRFiles + 'NR_0Ma_1000Ma_for_gplates.rot'

rotation_model = pygplates.RotationModel([rotation_file1,rotation_file2,rotation_file3,rotation_file4])


recon_label = 'M21-NNR'


ANCHOR_ID = int(0)

randomLons = []
randomLats = []

# Start with an empty list of features
features_to_modify = []
new_features=[]


NumberOfKimberlites=np.array([])
MyAges=np.array([])

inputFile=root+'/Tappe-et-al-2018-EPSL-Kimberlite_emplacement_ages-automatic-part1-with-locations-with-plate-IDs.gpml'

input_feature_collection = pygplates.FeatureCollection(inputFile)
# !rm Kimberlites_Appear*

# The temporal resolution in Myr.
step=-20
# The minimum age in Myr.
agewindow=int(abs(step)/2)
TotalKimb=0 # 
agemin=0
agemina=agemin

# The maximum age in Myr.
agemax=1010
agemaxa=agemax

# The list of files to merge.
filenames = []

# The list of features from all files to merge.
merged_features = []
mystep=0
# The name of the merged output file.

mergedOutputFile=root +'/Kimberlites/Kimberlites-%02d--PolyVertex-Merged-M21-NNR.gpml' %(mystep)



for age1 in np.arange(agemax,agemin,step):
    
    age2 = age1+step # min age for a single file, 980 at first iteration (age1=1000)

    age1f=float(age1) #1010 Ma in first iteration
    age2f=float(age2) #990 Ma in first iteration
    ageofVtk=age1-agewindow
    ageofVtkf=float(ageofVtk)


    
    # Start with an empty list of features that appear between age1 and age2 Ma, and put them at age1-10 Ma or age+10 Ma.
    features_to_modify = []
    
    for feature in input_feature_collection:
        name = feature.get_name()
        begin_time, end_time = feature.get_valid_time()
        #if begin_time > 0.0 and begin_time > end_time:
            #print name, begin_time, end_time
        if begin_time <= age1f and begin_time >= age2f:
            feature.set_valid_time(age1f, age2f)
            features_to_modify.append(feature)
        
        
    # Write the features that appear between age1 and age2 Ma to a new file.
    output_feature_collection = pygplates.FeatureCollection(features_to_modify)
#     pygplates.reconstruct(features_to_modify,rotation_model,outputFile1,age1f-10)
    
    anchor_plate_id=0
    Xr=[] 
    Yr=[] 
    Zr=[]
    Plat=[]
    Plon=[]
    # Write the Vtk's
    
    
    for feature in output_feature_collection:
#         print feature
        point = feature.get_geometry()
        PlateID = feature.get_reconstruction_plate_id()
        BirthTime=ageofVtkf
    
        # Get rotation for data point and reconstruct to 0Ma
        PlateRotation = rotation_model.get_rotation(BirthTime, PlateID, anchor_plate_id)
       
        reconstructed_point = PlateRotation * point
        
        reconstructed_point_degrees = reconstructed_point.to_lat_lon_point()
        
        Xr.append(reconstructed_point_degrees.get_longitude())
        Yr.append(reconstructed_point_degrees.get_latitude())

    
    
    NumberOfKimberlites=np.append(NumberOfKimberlites,len(Xr))
    
    MyAges=np.append(MyAges,ageofVtk)
    print(len(Xr)," Kimberlites at ",ageofVtk," Ma")
    
    TotalKimb=TotalKimb+len(Xr)
    count=0
    pointsToWrite=np.array([])
    
    spanning=0.35 # in degrees, (defines area of the hexahedron cap)
    theta=180
    phi=180
    

    for index in Xr: # index is longitude
        LatLon2D=np.array([index,Yr[count]])
        
         
        X=np.cos(LatLon2D[1]*np.pi/theta)*np.cos(LatLon2D[0]*np.pi/phi)
        Y=np.cos(LatLon2D[1]*np.pi/theta)*np.sin(LatLon2D[0]*np.pi/phi)
        Z=np.sin(LatLon2D[1]*np.pi/theta)

        
        pointsToWrite=np.append(pointsToWrite,[X,Y,Z])
        
        count=count+1
        
#     print len(pointsToWrite)
    pointsToWriteShaped=pointsToWrite.reshape(int(len(pointsToWrite)/3),3)
#         print pointsToWriteShaped, " : as ex."
    myconnectivity=[]

    for myindex in np.arange(0,len(pointsToWrite)/3,1):
        myCells=[0]*1
        myCells[0]=myindex
        myconnectivity.append(myCells)    
    
    
    myconnectivityNumpy=np.asarray(myconnectivity)


    cells = {"vertex": myconnectivityNumpy}
    
    rootsave="./Polyvertex-reconstructed-kimberlites/"   
    
    if len(pointsToWrite)<1:
        pointsToWrite=[[0.,0.,0.]]
        pointsToWriteAr=np.asarray(pointsToWrite)
        myconnectivityZero=[[0]]
        myconnectivityZeroAr=np.asarray(myconnectivityZero)
        cells2 = {"vertex": myconnectivityZeroAr} 
        meshio.write_points_cells(rootsave+"Kimberlites-M21-NNR_%02d_PolyVertex.vtu" %(mystep),pointsToWriteAr,cells2,file_format='vtu') 
    else:
        meshio.write_points_cells(rootsave+"Kimberlites-M21-NNR_%02d_PolyVertex.vtu" %(mystep),pointsToWriteShaped,cells,file_format='vtu')    
    
#     output_feature_collection.write(outputFile1)
    mystep=mystep+1
    


print(TotalKimb," is the total number of kimberlites processed ")

2  Kimberlites at  1000  Ma
1  Kimberlites at  980  Ma
1  Kimberlites at  960  Ma
1  Kimberlites at  940  Ma
1  Kimberlites at  920  Ma
0  Kimberlites at  900  Ma
1  Kimberlites at  880  Ma
0  Kimberlites at  860  Ma
2  Kimberlites at  840  Ma
1  Kimberlites at  820  Ma
8  Kimberlites at  800  Ma
1  Kimberlites at  780  Ma
2  Kimberlites at  760  Ma
0  Kimberlites at  740  Ma
2  Kimberlites at  720  Ma
2  Kimberlites at  700  Ma
4  Kimberlites at  680  Ma
3  Kimberlites at  660  Ma
5  Kimberlites at  640  Ma
2  Kimberlites at  620  Ma
5  Kimberlites at  600  Ma
19  Kimberlites at  580  Ma
12  Kimberlites at  560  Ma
23  Kimberlites at  540  Ma
16  Kimberlites at  520  Ma
15  Kimberlites at  500  Ma
11  Kimberlites at  480  Ma
8  Kimberlites at  460  Ma
10  Kimberlites at  440  Ma
4  Kimberlites at  420  Ma
11  Kimberlites at  400  Ma
26  Kimberlites at  380  Ma
23  Kimberlites at  360  Ma
7  Kimberlites at  340  Ma
8  Kimberlites at  320  Ma
2  Kimberlites at  300  Ma
5  Kimberlites at


# Reconstruct Kimberlites, Export them as Pseudo Drillholes to visualize in ParaView (*vtu files) and GPlates (NetCDF files)

In [22]:
def find(lst, a, b):
    return [i for i, x in enumerate(lst) if x<a and x>b]


root='.'

NNRFiles=root+'/NNR_InFiles/'

rotation_file1 = NNRFiles + '1000-410_rotations-NNR.rot'
rotation_file2 = NNRFiles + 'Global_EB_410-250Ma_GK07_2017-NNR.rot'
rotation_file3 = NNRFiles + 'Global_EB_250-0Ma_GK07_2017-NNR.rot'
rotation_file4 = NNRFiles + 'NR_0Ma_1000Ma_for_gplates.rot'

rotation_model = pygplates.RotationModel([rotation_file1,rotation_file2,rotation_file3,rotation_file4])


recon_label = 'M21-NNR'


ANCHOR_ID = int(0)


randomLons = []
randomLats = []

# Start with an empty list of features
features_to_modify = []
new_features=[]


NumberOfKimberlites=np.array([])
MyAges=np.array([])

inputFile='/Volumes/Accelsior4M2/ada2/KimberlitesReconstructed/Tappe-et-al-2018-EPSL-Kimberlite_emplacement_ages-automatic-part1-with-locations-with-plate-IDs.gpml'

input_feature_collection = pygplates.FeatureCollection(inputFile)
# !rm Kimberlites_Appear*

# The temporal resolution in Myr.
step=-20
# The minimum age in Myr.
agewindow=int(abs(step)/2)
TotalKimb=0 # 
agemin=0
agemina=agemin

# The maximum age in Myr.
agemax=1010
agemaxa=agemax

# The list of files to merge.
filenames = []

# The list of features from all files to merge.
merged_features = []
mystep=0
# The name of the merged output file.


for age1 in np.arange(agemax,agemin,step):
    
    age2 = age1+step # min age for a single file, 980 at first iteration (age1=1000)

    age1f=float(age1) #1010 Ma in first iteration
    age2f=float(age2) #990 Ma in first iteration
    ageofVtk=age1-agewindow
    ageofVtkf=float(ageofVtk)
#     age3f=float(age3)
#     age4f=float(age4)
    
#     print age1f, age2f, age3f, age4f    
    
#     outputFile1=here + 'Kimberlites_Appear_between_%dMa_and_%dMa_mod.gpml' %(age1,age2)
#     print outputFile1
    
    filenames.append(outputFile1)
    
    # Start with an empty list of features that appear between age1 and age2 Ma, and put them at age1-10 Ma or age+10 Ma.
    features_to_modify = []
    
    for feature in input_feature_collection:
        name = feature.get_name()
        begin_time, end_time = feature.get_valid_time()
        
        if begin_time <= age1f and begin_time >= age2f:
            feature.set_valid_time(age1f, age2f)
            features_to_modify.append(feature)
        
        
    # Write the features that appear between age1 and age2 Ma to a new file.
    output_feature_collection = pygplates.FeatureCollection(features_to_modify)
#     pygplates.reconstruct(features_to_modify,rotation_model,outputFile1,age1f-10)
    
    anchor_plate_id=0
    Xr=[] 
    Yr=[] 
    Zr=[]
    Plat=[]
    Plon=[]
    # Write the Vtk's
    
    
    for feature in output_feature_collection:
#         print feature
        point = feature.get_geometry()
        PlateID = feature.get_reconstruction_plate_id()
        BirthTime=ageofVtkf
    
        # Get rotation for data point and reconstruct to 0Ma
        PlateRotation = rotation_model.get_rotation(BirthTime, PlateID, anchor_plate_id)
       
        reconstructed_point = PlateRotation * point
        
        reconstructed_point_degrees = reconstructed_point.to_lat_lon_point()
        
        Xr.append(reconstructed_point_degrees.get_longitude())
        Yr.append(reconstructed_point_degrees.get_latitude())

    
    
    NumberOfKimberlites=np.append(NumberOfKimberlites,len(Xr))
    
    MyAges=np.append(MyAges,ageofVtk)
#     print(len(Xr)," Kimberlites at ",ageofVtk," Ma")
    TotalKimb=TotalKimb+len(Xr)
    count=0
    pointsToWrite=np.array([])
    
    spanning=0.35 # in degrees, (defines area of the hexahedron cap)

    WholeMantle=0.1
#     UpperMantle=0.896481990814209
    UpperMantle=0.8
    
    theta=180
    phi=180
    
    try: ncfile.close() 
    except: pass
    rootSaveNetCDF="./Drillholes-at-reconstructed-kimberlites-GPlates/"
    os.mkdir(rootSaveNetCDF+'NetCDFdata/'+str(ageofVtk))
    ncfile = Dataset(rootSaveNetCDF+'NetCDFdata/'+str(ageofVtk)+'/Drillholes-at-reconstructed-kimberlites-M21NNR-'+str(ageofVtkf)+'Ma-0.nc',mode='w',format='NETCDF4_CLASSIC') 
#     print(ncfile)
    lat_dim = ncfile.createDimension('lat', 721)     # latitude axis
    lon_dim = ncfile.createDimension('lon', 1441)    # longitude axis
    lat = ncfile.createVariable('lat', np.float32, ('lat',))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lon = ncfile.createVariable('lon', np.float32, ('lon',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    temp = ncfile.createVariable('temp',np.float32,('lat','lon'))
    temp.standard_name = 'KimberliteEmplacement' 
#     print(temp)

    nlats = len(lat_dim); nlons = len(lon_dim); ntimes=1;
    lat[:] = -90. + (180./(nlats-1))*np.arange(nlats) # south pole to north pole
    lon[:] = (360./(nlons-1))*np.arange(nlons) # Greenwich meridian eastward
    
    dataLat=np.zeros(lat.shape)
    
    dataLon=np.zeros(lon.shape)
    
    dataArrayForNetCDFSON=np.zeros((721,1441))
    for index in Xr: # index is longitude
        LatLon2D=np.array([index,Yr[count]])
        
        dataLat=np.zeros(lat.shape)
    
        dataLon=np.zeros(lon.shape)
        
        if (LatLon2D[0]<0):
            lonToUse=LatLon2D[0]+360.0
        else:
            lonToUse=LatLon2D[0]
                
        
        indexesLon=find(list(lon),lonToUse+spanning,lonToUse-spanning)
        dataLon[indexesLon]=1.0
        
        indexesLat=find(list(lat),LatLon2D[1]+spanning,LatLon2D[1]-spanning)
        dataLat[indexesLat]=1.0
        
        dataArrayForNetCDF=np.dot(np.reshape(dataLat,(721,1)),np.reshape(dataLon,(1,1441)))
        
        Lat_minus1_Lon_minus1=np.array([index-spanning,Yr[count]-spanning])
        Lat_minus1_Lon_plus1=np.array([index+spanning,Yr[count]-spanning])
        Lat_plus1_Lon_plus1=np.array([index+spanning,Yr[count]+spanning])
        Lat_plus1_Lon_minus1=np.array([index-spanning,Yr[count]+spanning])

        X=np.cos(LatLon2D[1]*np.pi/theta)*np.cos(LatLon2D[0]*np.pi/phi)
        Y=np.cos(LatLon2D[1]*np.pi/theta)*np.sin(LatLon2D[0]*np.pi/phi)
        Z=np.sin(LatLon2D[1]*np.pi/theta)

        X_0=np.cos(Lat_minus1_Lon_minus1[1]*np.pi/theta)*np.cos(Lat_minus1_Lon_minus1[0]*np.pi/phi)
        Y_0=np.cos(Lat_minus1_Lon_minus1[1]*np.pi/theta)*np.sin(Lat_minus1_Lon_minus1[0]*np.pi/phi)
        Z_0=np.sin(Lat_minus1_Lon_minus1[1]*np.pi/theta)
         
        X_1=np.cos(Lat_minus1_Lon_plus1[1]*np.pi/theta)*np.cos(Lat_minus1_Lon_plus1[0]*np.pi/phi)
        Y_1=np.cos(Lat_minus1_Lon_plus1[1]*np.pi/theta)*np.sin(Lat_minus1_Lon_plus1[0]*np.pi/phi)
        Z_1=np.sin(Lat_minus1_Lon_plus1[1]*np.pi/theta)
        
        X_2=np.cos(Lat_plus1_Lon_plus1[1]*np.pi/theta)*np.cos(Lat_plus1_Lon_plus1[0]*np.pi/phi)
        Y_2=np.cos(Lat_plus1_Lon_plus1[1]*np.pi/theta)*np.sin(Lat_plus1_Lon_plus1[0]*np.pi/phi)
        Z_2=np.sin(Lat_plus1_Lon_plus1[1]*np.pi/theta)
        
        X_3=np.cos(Lat_plus1_Lon_minus1[1]*np.pi/theta)*np.cos(Lat_plus1_Lon_minus1[0]*np.pi/phi)
        Y_3=np.cos(Lat_plus1_Lon_minus1[1]*np.pi/theta)*np.sin(Lat_plus1_Lon_minus1[0]*np.pi/phi)
        Z_3=np.sin(Lat_plus1_Lon_minus1[1]*np.pi/theta)
        
        pointsToWrite=np.append(pointsToWrite,[X_0,Y_0,Z_0])
        pointsToWrite=np.append(pointsToWrite,[X_1,Y_1,Z_1])
        pointsToWrite=np.append(pointsToWrite,[WholeMantle*X_1,WholeMantle*Y_1,WholeMantle*Z_1])
        pointsToWrite=np.append(pointsToWrite,[WholeMantle*X_0,WholeMantle*Y_0,WholeMantle*Z_0])
        pointsToWrite=np.append(pointsToWrite,[X_3,Y_3,Z_3])
        pointsToWrite=np.append(pointsToWrite,[X_2,Y_2,Z_2])
        pointsToWrite=np.append(pointsToWrite,[WholeMantle*X_2,WholeMantle*Y_2,WholeMantle*Z_2])
        pointsToWrite=np.append(pointsToWrite,[WholeMantle*X_3,WholeMantle*Y_3,WholeMantle*Z_3])
        count=count+1
        
        dataArrayForNetCDFSON=dataArrayForNetCDF+dataArrayForNetCDFSON
#     print len(pointsToWrite)
    pointsToWriteShaped=pointsToWrite.reshape(int(len(pointsToWrite)/3),3)
#         print pointsToWriteShaped, " : as ex."
    myconnectivity=[]

    for myindex in np.arange(0,len(pointsToWrite)/3,8):
        myCells=[0]*8
        myCells[0]=myindex
        myCells[1]=myindex+1
        myCells[2]=myindex+2
        myCells[3]=myindex+3
        myCells[4]=myindex+4
        myCells[5]=myindex+5
        myCells[6]=myindex+6
        myCells[7]=myindex+7
        myconnectivity.append(myCells)    
        myconnectivityNumpy=np.asarray(myconnectivity)


    cells = {"hexahedron": myconnectivityNumpy}
    
    rootsaveForKimb="./Drillholes-at-reconstructed-kimberlites-ParaView/"
    if len(pointsToWrite)<1:
        pointsToWrite=[[0.,0.,0.],[0.01,0.01,0.01]]
        pointsToWriteAr=np.asarray(pointsToWrite)
        myconnectivityZero=[[0,1]]
        myconnectivityZeroAr=np.asarray(myconnectivityZero)
        cells2 = {"line": myconnectivityZeroAr} 
        meshio.write_points_cells(rootsaveForKimb+"Drillholes-at-reconstructed-kimberlites-M21-NNR_%02d.vtu" %(mystep),pointsToWriteAr,cells2,file_format='vtu') 
    else:
        meshio.write_points_cells(rootsaveForKimb+"Drillholes-at-reconstructed-kimberlites-M21-NNR_%02d.vtu" %(mystep),pointsToWriteShaped,cells,file_format='vtu')    
    
    mystep=mystep+1
    
#     dataArrayForNetCDF=np.dot(np.reshape(dataLat,(721,1)),np.reshape(dataLon,(1,1441)))
#     dataArrayForNetCDF=np.cross(np.reshape(dataLat,(721,1)),np.reshape(dataLon,(1,1441)))    
    temp[:,:] = dataArrayForNetCDFSON # Appends data along unlimited dimension
    ncfile.close();
     
#     STOP

print(TotalKimb," is the total number of kimberlites processed ")

967  is the total number of kimberlites processed 
